### V2 - a working model emerges

This version uses base-learner-6 .pkl/.pth

Which is the standard used in app

The most important elemnt is adding Crop to the dataloader initially which works its way into the learner's preproc  routine.

Technichally we're supposed use `MyResizeDeterm` as the after_tem tfms (because this is how this model was trained), but that is custom code that hasn't been ported. You can see the results are quite good on preds below.

prev section:

------

We need to load a pth torch object instead of fastai learner (from pkl).

This is because pickle preserve of the python object breaks when loading with new import name.


10.14.2020

In [1]:
import torch
from fastai.vision.all import *

In [2]:
def piece_class_parse(fn): 
    fn = fn.split('_')[1]
    fn = fn.split('.')[0]
    return fn

### Build Learn + Train_DL

In [4]:
_train_path = Path('../../../rf-chess-data/cropped_v1/')

train_fnames = get_image_files(_train_path)

In [ ]:
# Crop = MyResizeDeterm(128, _resize_method, pad_mode=_pad_mode)

In [24]:
Crop = Resize(128, ResizeMethod.Pad, pad_mode='reflection')

In [25]:
train_dl = ImageDataLoaders.from_name_func(
                    _train_path, 
                    train_fnames,
                    valid_pct=.2, 
                    label_func=piece_class_parse, 
                    item_tfms=Crop,
#                     batch_tfms=Augs,
#                     bs=_bs,
                    # num_workers=0,
                    )

In [26]:
learn = cnn_learner(train_dl, resnet50, metrics=[], normalize=False)


### Pred, No Model

In [27]:
import os
d = '../../../other-chess-data/regulation-pieces-1//crops/'
fns = os.listdir(d)
img_fns = [d + fn for fn in fns[:10]]

In [28]:
naive_preds = [learn.predict(e)[0] for e in img_fns]

### Load StateDict from pth

In [29]:
PATH = '../models/base-learner-6.pth'

model = learn.model

model.load_state_dict(torch.load(PATH))

model.eval();

In [30]:
learn.model = model

### View Predictions

In [31]:
actuals = [Path(e).name for e in img_fns]

In [32]:
preds = [learn.predict(e)[0] for e in img_fns]

In [33]:
[(a,b,c) for a,b,c in zip(actuals, preds, naive_preds)]

[('00000_white-king.jpg', 'white-king', 'white-queen'),
 ('00001_black-rook.jpg', 'black-rook', 'black-pawn'),
 ('00002_black-knight.jpg', 'black-knight', 'black-pawn'),
 ('00003_white-knight.jpg', 'black-knight', 'white-pawn'),
 ('00004_white-pawn.jpg', 'white-pawn', 'white-queen'),
 ('00005_black-pawn.jpg', 'black-pawn', 'white-king'),
 ('00006_white-bishop.jpg', 'white-bishop', 'white-queen'),
 ('00007_white-rook.jpg', 'white-rook', 'black-bishop'),
 ('00008_white-queen.jpg', 'white-queen', 'black-king'),
 ('00009_black-bishop.jpg', 'black-bishop', 'white-queen')]

### Use Get Preds
Quite good, even though we're probably using training data here to test

In [34]:
gp = learn.get_preds()

In [35]:
len(gp)

2

In [40]:
gp[1].shape

torch.Size([574])

In [41]:
gp[0].argmax(1).shape

torch.Size([574])

In [42]:
from sklearn.metrics import accuracy_score

In [43]:
accuracy_score(gp[1], gp[0].argmax(1))

0.9965156794425087